In [39]:
from llama_cpp import Llama
import pandas as pd
import json
import random
from sklearn.metrics import classification_report

In [57]:
df = pd.read_json('data/llm_task1_hetero.json', orient="records")
df.head()

,id,text,label
0,het_857,Those whose child is going through a'phase'may...,1
1,het_368,Bureaucrats cut red tape-- lengthwise.,0
2,het_1204,If they say you look good in glasses that woul...,1
3,het_1043,"Nearing the end of his sentence, the convicted...",1
4,het_1098,A logger went to his doctor because of a probl...,1


In [59]:
test = df
len(test)

445

In [85]:
PATH = 'models/mistral-7b-openorca.Q6_K.gguf'
llm = Llama(model_path=PATH, n_gpu_layers=75, n_ctx=2048)

llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from models/mistral-7b-openorca.Q6_K.gguf (version GGUF V2)
llama_model_loader: - tensor    0:                token_embd.weight q6_K     [  4096, 32002,     1,     1 ]
llama_model_loader: - tensor    1:              blk.0.attn_q.weight q6_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    2:              blk.0.attn_k.weight q6_K     [  4096,  1024,     1,     1 ]
llama_model_loader: - tensor    3:              blk.0.attn_v.weight q6_K     [  4096,  1024,     1,     1 ]
llama_model_loader: - tensor    4:         blk.0.attn_output.weight q6_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_gate.weight q6_K     [  4096, 14336,     1,     1 ]
llama_model_loader: - tensor    6:              blk.0.ffn_up.weight q6_K     [  4096, 14336,     1,     1 ]
llama_model_loader: - tensor    7:            blk.0.ffn_down.weight q6_K     [ 14336,  4096,     1,     1 ]

In [44]:
def is_json(strJson):
  try:
    json.loads(strJson)
  except ValueError as e:
    return False
  return True

In [45]:
def generate(prompt, attempt):
  if attempt == 1:
    temp = 0.2
  else:
    temp = round(random.uniform(0.3, 0.6), 1)
  output = llm(prompt, max_tokens=-1, temperature=temp)
  return output['choices'][0]['text']

def parseResult(jsonStr, row):
  if not is_json(jsonStr.strip()):
    return None
  else:
    parsedJson = json.loads(jsonStr.strip())
    return parsedJson

### Task 1: Pun Detection

In [43]:
prompt = f''' ### INSTRUCTION:
You are to classify whether a given sentence is a pun based on the following criteria:
1. Ambiguity -  there exists a word in the sentence that has two similarly likely interpretations.
2. Distinctiveness - the two interpretations are very different from each other i.e. how distinct are the words semantically related to the two interpretations from each other. There needs to be at least one different word in the set of words supporting each interpretation.\n
For a given sentence to be a pun, it should satisfy BOTH criteria - Ambiguity and Distinctiveness.\n
The final output is either true or false where true means that the sentence is a pun.\n
It is possible that a sentence does not have a word that is ambiguous. The result in this case is false.\n
It is possible that a sentence has Ambiguity but no Distinctiveness. The result in this case is false.\n
Here are three examples of the cases you will encounter:
1. An example with both ambiguity and distinctiveness - 
The magician got so mad that he pulled his hare out.
{{
  "output": true,
  "ambiguity": "The pun word 'hare' supports two plausible interpretations of 'hare' meaning a rabbit and 'hair' meaning human hair.",
  "distinctiveness": "In the given sentence, the words 'magician' relates to 'hare' while 'pulled' refers to the second interpretation of 'hair'. Both of these are distinct where one refers to a magician's animal while the other refers to an action done in anger which is pulling your hair."
}}
2. An example with only ambiguity -
I went to the bank.
{{
  "output": false,
  "ambiguity": "The word 'bank' does have ambiguity here where it supports two plausible interpretations of bank as in a financial institution and bank as in the banks of a river.",
  "distinctiveness": "There are no other words in the sentence that provide distinctiveness to the two interpretations, the sentence is not a pun."
}}
3. An example with neither ambiguity nor distinctiveness -
Let us go home.
{{
  "output": false,
  "ambiguity": "There is no ambiguous word in the sentence.",
  "distinctiveness": "Not applicable"
}}
Identify whether the input sentence is a pun and explain the result based on ambiguity and distinctiveness in valid JSON format. Generate a response in the form of a valid JSON object with three keys: output, ambiguity and distinctiveness.\n
### INPUT:\n
'''

In [60]:
results = []
i = 0
for _, row in test.iterrows():
  prepared_prompt = prompt + row['text'] + "\n### OUTPUT:\n"
  json_out = ''
  attempt = 0
  while True:
    attempt += 1
    json_out = generate(prepared_prompt, attempt)
    parsedJson = parseResult(json_out, row)
    print(f"{i} -- {attempt}")
    if parsedJson is not None:
      parsedJson['id'] = row['id']
      parsedJson['text'] = row['text']
      results.append(parsedJson)
      i+=1
      break


Llama.generate: prefix-match hit


0 -- 1



llama_print_timings:        load time =    4274.02 ms
llama_print_timings:      sample time =      15.16 ms /   116 runs   (    0.13 ms per token,  7653.23 tokens per second)
llama_print_timings: prompt eval time =    2160.70 ms /    25 tokens (   86.43 ms per token,    11.57 tokens per second)
llama_print_timings:        eval time =    6151.46 ms /   115 runs   (   53.49 ms per token,    18.69 tokens per second)
llama_print_timings:       total time =    8529.95 ms
Llama.generate: prefix-match hit


1 -- 1



llama_print_timings:        load time =    4274.02 ms
llama_print_timings:      sample time =      15.18 ms /   128 runs   (    0.12 ms per token,  8431.59 tokens per second)
llama_print_timings: prompt eval time =     192.01 ms /    17 tokens (   11.29 ms per token,    88.54 tokens per second)
llama_print_timings:        eval time =    6748.20 ms /   127 runs   (   53.14 ms per token,    18.82 tokens per second)
llama_print_timings:       total time =    7147.94 ms
Llama.generate: prefix-match hit

llama_print_timings:        load time =    4274.02 ms
llama_print_timings:      sample time =      17.39 ms /   119 runs   (    0.15 ms per token,  6844.98 tokens per second)
llama_print_timings: prompt eval time =     199.17 ms /    22 tokens (    9.05 ms per token,   110.46 tokens per second)
llama_print_timings:        eval time =    6368.90 ms /   118 runs   (   53.97 ms per token,    18.53 tokens per second)
llama_print_timings:       total time =    6805.53 ms
Llama.generate: prefix-

...


llama_print_timings:        load time =    4274.02 ms
llama_print_timings:      sample time =      12.36 ms /   127 runs   (    0.10 ms per token, 10273.42 tokens per second)
llama_print_timings: prompt eval time =     198.77 ms /    21 tokens (    9.47 ms per token,   105.65 tokens per second)
llama_print_timings:        eval time =    6760.77 ms /   126 runs   (   53.66 ms per token,    18.64 tokens per second)
llama_print_timings:       total time =    7122.79 ms
Llama.generate: prefix-match hit


443 -- 1



llama_print_timings:        load time =    4274.02 ms
llama_print_timings:      sample time =       3.87 ms /    39 runs   (    0.10 ms per token, 10082.73 tokens per second)
llama_print_timings: prompt eval time =     207.12 ms /    28 tokens (    7.40 ms per token,   135.19 tokens per second)
llama_print_timings:        eval time =    2031.75 ms /    38 runs   (   53.47 ms per token,    18.70 tokens per second)
llama_print_timings:       total time =    2288.93 ms
Llama.generate: prefix-match hit


444 -- 1



llama_print_timings:        load time =    4274.02 ms
llama_print_timings:      sample time =       3.85 ms /    40 runs   (    0.10 ms per token, 10395.01 tokens per second)
llama_print_timings: prompt eval time =     190.95 ms /    16 tokens (   11.93 ms per token,    83.79 tokens per second)
llama_print_timings:        eval time =    2083.72 ms /    39 runs   (   53.43 ms per token,    18.72 tokens per second)
llama_print_timings:       total time =    2324.77 ms


In [61]:
results_df = pd.DataFrame(results, columns=['id', 'text', 'output', 'ambiguity', 'distinctiveness',])
results_df.to_json('outputs-task1-hetero-openocra7b.json', orient="records")
len(results_df)

445

In [62]:
results_df['output'] = results_df['output'].replace({True: 1, False: 0})

In [63]:
y_true = [int(x) for x in list(test['label'])]
y_pred = [int(x) for x in list(results_df['output'])]

In [64]:
print(classification_report(y_true, y_pred, digits=4))

              precision    recall  f1-score   support

           0     0.4358    0.7480    0.5507       127
           1     0.8590    0.6132    0.7156       318

    accuracy                         0.6517       445
   macro avg     0.6474    0.6806    0.6332       445
weighted avg     0.7382    0.6517    0.6685       445



### Task 2 and 3: Pun Location & Interpretation

In [82]:
def clean_sentence(text):
  res = text.replace(" ' ' ", '" ')
  res = res.replace(" ' ", "'")
  res = res.replace(" , ", ", ")
  res = res.replace(" . ", ". ")
  res = res.replace(" .", ". ")
  res = res.replace(" ! ", "! ")
  res = res.replace(" !", "! ")
  res = res.replace(" ? ", "? ")
  res = res.replace(" ?", "? ")
  res = res.replace(" - ", "-")
  return res.strip()

In [129]:
df = pd.read_json('data/llm_task2_hetero.json', orient="records")
df['sentence'] = df['sentence'].apply(clean_sentence)
df = df.rename(columns={'sentence': 'text'})
df.head()

,text,pun_word
0,In some countries there is a lot of Manuel labor.,Manuel
1,Someone who attempts long distance fraud is kn...,phony
2,"In the delivery room, the unborn infant expect...",berth
3,Beautifully manicured lawns are highly sod after.,sod
4,"I saw something similar to moss the other day,...",lichen


In [130]:
test = df
len(test)

318

In [131]:
prompt = f''' ### INSTRUCTION:
You are to identify the pun word in a given sentence based on the following criteria:
1. Ambiguity -  there exists a word in the sentence that has two similarly likely interpretations.
2. Distinctiveness - the two interpretations are very different from each other i.e. how distinct are the words semantically related to the two interpretations from each other. There needs to be at least one different word in the set of words supporting each interpretation.\n
For a word to be a pun word, it should satisfy BOTH criteria - Ambiguity and Distinctiveness.\n
It is guaranteed that the sentence has a pun word and hence the selected pun word must be from the sentence in the same form as it appears.\n
Also, list all the words that realize each interpretation of the pun word. If there are multiple words that support an interpretation, separate them with a semicolon.\n
Here are two examples of the cases you will encounter:
1. An example with heterographic puns (words that sound similar) - 
The magician got so mad that he pulled his hare out.
{{
  "pun_word": "hare",
  "target_word": "hair",
  "source_sense": "magician",
  "target_sense": "angry;pulled"
}}

In the above sentence, the pun word 'hare' supports two plausible interpretations of 'hare' meaning a rabbit and 'hair' meaning human hair. In the given sentence, the words 'magician' relates to 'hare' while 'angry' and 'pulled' refer to the second interpretation of 'hair'.

2. An example with homographic puns (same words with different senses) -
I used to be a banker but I lost interest.
{{
  "pun_word": "interest",
  "target_word": "interest",
  "source_sense": "banker",
  "target_sense": "used;to;be"
}}

In the above sentence, the pun word 'interest' supports two distinct interpretations of interest meaning 'financial interest' and interest meaning 'desire to do something'. The words 'banker' realize the first sense of financial interest while the words 'used', 'to', 'be' refer to the second sense.

Identify the pun word in the input sentence is a pun and list the target word and the supporting words for each sense the result based on ambiguity and distinctiveness in valid JSON format. Do not explain. Generate a response in the form of a valid JSON object with four keys: pun_word, target_word, source_sense and target_sense.\n
### INPUT:\n
'''

In [132]:
results = []
i = 0
for _, row in test.iterrows():
  prepared_prompt = prompt + row['text'] + "\n### OUTPUT:\n"
  json_out = ''
  attempt = 0
  while True:
    attempt += 1
    json_out = generate(prepared_prompt, attempt)
    parsedJson = parseResult(json_out, row)
    print(f"{i} -- {attempt}")
    if parsedJson is not None:
      parsedJson['text'] = row['text']
      results.append(parsedJson)
      i+=1
      break

Llama.generate: prefix-match hit


0 -- 1



llama_print_timings:        load time =    5882.46 ms
llama_print_timings:      sample time =       7.92 ms /    52 runs   (    0.15 ms per token,  6561.51 tokens per second)
llama_print_timings: prompt eval time =    6128.85 ms /    17 tokens (  360.52 ms per token,     2.77 tokens per second)
llama_print_timings:        eval time =    2709.51 ms /    51 runs   (   53.13 ms per token,    18.82 tokens per second)
llama_print_timings:       total time =    8960.64 ms
Llama.generate: prefix-match hit


1 -- 1



llama_print_timings:        load time =    5882.46 ms
llama_print_timings:      sample time =       5.39 ms /    56 runs   (    0.10 ms per token, 10391.54 tokens per second)
llama_print_timings: prompt eval time =     194.59 ms /    20 tokens (    9.73 ms per token,   102.78 tokens per second)
llama_print_timings:        eval time =    2906.99 ms /    55 runs   (   52.85 ms per token,    18.92 tokens per second)
llama_print_timings:       total time =    3177.65 ms
Llama.generate: prefix-match hit


...


llama_print_timings:        load time =    5882.46 ms
llama_print_timings:      sample time =       5.30 ms /    55 runs   (    0.10 ms per token, 10375.40 tokens per second)
llama_print_timings: prompt eval time =     198.78 ms /    22 tokens (    9.04 ms per token,   110.67 tokens per second)
llama_print_timings:        eval time =    2857.47 ms /    54 runs   (   52.92 ms per token,    18.90 tokens per second)
llama_print_timings:       total time =    3130.26 ms
Llama.generate: prefix-match hit


316 -- 1



llama_print_timings:        load time =    5882.46 ms
llama_print_timings:      sample time =       6.41 ms /    62 runs   (    0.10 ms per token,  9675.41 tokens per second)
llama_print_timings: prompt eval time =     210.69 ms /    30 tokens (    7.02 ms per token,   142.39 tokens per second)
llama_print_timings:        eval time =    3232.03 ms /    61 runs   (   52.98 ms per token,    18.87 tokens per second)
llama_print_timings:       total time =    3531.06 ms
Llama.generate: prefix-match hit


317 -- 1



llama_print_timings:        load time =    5882.46 ms
llama_print_timings:      sample time =       6.08 ms /    62 runs   (    0.10 ms per token, 10197.37 tokens per second)
llama_print_timings: prompt eval time =     188.53 ms /    28 tokens (    6.73 ms per token,   148.52 tokens per second)
llama_print_timings:        eval time =    3231.61 ms /    61 runs   (   52.98 ms per token,    18.88 tokens per second)
llama_print_timings:       total time =    3504.93 ms


In [133]:
results_df = pd.DataFrame(results, columns=['text', 'pun_word', 'target_word', 'source_sense', 'target_sense'])
results_df.to_json('outputs-task2-hetero-openocra7b.json', orient="records")
len(results_df)

318

In [134]:
def cprf(true, pred):
  correct = (true['pun_word'].str.strip().str.lower() == pred['pun_word'].str.strip().str.lower()).sum()
  coverage = len(pred) / len(true)
  precision = correct / len(pred)
  recall = correct / len(true)
  f1 = (2 * precision * recall) / (precision + recall)
  return coverage, precision, recall, f1

In [136]:
c, p, r, f1 = cprf(test, results_df)
print(f"Coverage: {c:.4f}")
print(f"Precision: {p:.4f}")
print(f"Recall: {r:.4f}")
print(f"F1-Score: {f1:.4f}")

Coverage: 1.0000
Precision: 0.6258
Recall: 0.6258
F1-Score: 0.6258
